In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras import utils
import keras_tuner as kt

utils.set_random_seed(42)

In [ ]:
# Define Parameters
LOOKBACK = 24
HORIZON = 24
N_SPLITS = 4
EPOCHS = 10
scaler = StandardScaler()

# Funcs

In [ ]:
# Time series split function (Expanding Window)
def time_series_split(df, n_splits=N_SPLITS, test_size=0.2):
    df = df.sort_values('timestamp')
    test_split_index = int(len(df) * (1 - test_size))
    train_val_df = df.iloc[:test_split_index]
    test_df = df.iloc[test_split_index:]

    tscv = TimeSeriesSplit(n_splits=n_splits)
    splits = [(train_val_df.iloc[train_index], train_val_df.iloc[val_index]) for train_index, val_index in tscv.split(train_val_df)]
    return splits, test_df

In [ ]:
# Sequence creation for univariate time series
def create_sequences(df, lookback=LOOKBACK, horizon=HORIZON):
    X, y, anomaly, cell_id = [], [], [], []

    # Loop through each unique cell in the dataset
    for cell in df['cell'].unique():
        # Filter the dataframe for the current cell only
        cell_df = df[df['cell'] == cell]

        # Generate sequences within this cell's data
        for i in range(lookback, len(cell_df) - horizon + 1):
            # Lookback sequence for minRSSI only (univariate)
            X_seq = cell_df.iloc[i - lookback:i][['minRSSI']].values
            # Target horizon sequence for minRSSI
            y_seq = cell_df.iloc[i:i + horizon]['minRSSI'].values
            # Anomaly sequences for later evaluation
            anomaly_seq = cell_df.iloc[i:i + horizon]['anomaly'].values
            # Cell ID for each sequence
            cell_seq = cell_df.iloc[i:i + horizon]['cell'].values

            # Append sequences to output lists
            X.append(X_seq)
            y.append(y_seq)
            anomaly.append(anomaly_seq)
            cell_id.append(cell_seq)

    # Convert lists to numpy arrays for model input
    return np.array(X), np.array(y), np.array(anomaly), np.array(cell_id)

In [ ]:
def tune_hp_hyperband_lstm(X_train, y_train, max_epochs=EPOCHS):

    def build_tunable_lstm(hp):
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1], 1)))

        # Add LSTM layers with recurrent dropout
        for i in range(hp.Int('num_lstm_layers', min_value=1, max_value=4)):
            model.add(LSTM(
                units=hp.Choice('units', values=[32, 64, 128, 256]),
                activation=hp.Choice('activation', values=['relu', 'tanh']),
                return_sequences=True if i < hp.get('num_lstm_layers') - 1 else False,
                kernel_regularizer=l2(hp.Choice('l2_regularizer', values=[1e-2, 1e-3, 1e-4])),
                recurrent_dropout=hp.Float(f'recurrent_dropout_{i}', min_value=0.2, max_value=0.5, step=0.1)
            ))

        # Add dense layers
        for j in range(hp.Int('num_dense_layers', min_value=1, max_value=3)):
            model.add(Dense(units=hp.Choice(f'dense_units_{j}', values=[32, 64, 128, 256])))

            # Optional dense layer dropout
            if hp.Boolean(f'use_dense_dropout_{j}'):
                dense_dropout_rate = hp.Float(f'dense_dropout_rate_{j}', min_value=0.1, max_value=0.3, step=0.1)
                model.add(Dropout(dense_dropout_rate))

        model.add(Dense(HORIZON))

        # Compile model with fixed optimizer (Adam) and tunable loss function
        model.compile(
            optimizer='adam',
            loss=hp.Choice('loss', values=['mse', 'mae'])
        )

        # Define batch size as a tunable hyperparameter
        batch_size = hp.Choice('batch_size', [16, 32, 64, 128])

        return model

    # Hyperband tuner instance
    tuner = kt.Hyperband(
        hypermodel=build_tunable_lstm,
        objective='val_loss',
        max_epochs=max_epochs,
        factor=3,
        directory='/content/drive/MyDrive/Thesis/Thesis/lstm/univar_tuning',
        project_name='lstm_tuning'
    )

    # Fit Hyperband tuner to training data
    tuner.search(X_train, y_train,
                 epochs=max_epochs,
                 batch_size=hp.Choice('batch_size', values=[16, 32, 64, 128]),
                 validation_split=0.2,
                 verbose=1)

    # Get best model and hyperparameters
    best_model = tuner.get_best_models(num_models=1)[0]
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0].values

    return best_model, best_params

# Prepare data

In [ ]:
imp_folder = os.getenv("DATA_PATH", "./default_data_path/")
exp_folder = os.getenv("MODEL_PATH", "./default_model_path/")

df = pd.read_csv(imp_folder + 'cell_undersampled_1.csv')
df = df[['timestamp', 'cell', 'minRSSI', 'anomaly']]

print(df.shape)
df.head()

In [ ]:
splits, test_set = time_series_split(df, 4)

for i, (train, val) in enumerate(splits):
    print(f"Split {i + 1}:")
    print(f"  Train set shape: {train.shape}")
    print(f"  Validation set shape: {val.shape}")

print(f"Test set shape: {test_set.shape}")

Split 1:
  Train set shape: (151460, 4)
  Validation set shape: (151459, 4)
Split 2:
  Train set shape: (302919, 4)
  Validation set shape: (151459, 4)
Split 3:
  Train set shape: (454378, 4)
  Validation set shape: (151459, 4)
Split 4:
  Train set shape: (605837, 4)
  Validation set shape: (151459, 4)
Test set shape: (189324, 4)


# Run the tuning

In [ ]:
all_best_params = []

for i, (train_set, val_set) in enumerate(splits):
    print(f"Processing split {i + 1}/{N_SPLITS}")

    # Preprocess the data (fit scaler on train, transform on train and val)
    train_set['minRSSI'] = scaler.fit_transform(train_set[['minRSSI']])
    val_set['minRSSI'] = scaler.transform(val_set[['minRSSI']])

    # Create sequences for train and validation sets
    X_train, y_train, _, _ = create_sequences(train_set)
    X_val, y_val, _, _ = create_sequences(val_set)

    # Reshape X for 1D CNN (samples, timesteps, features)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

    # Tune and train the model
    best_model, best_params = tune_hp_hyperband_lstm(X_train, y_train)
    all_best_params.append(best_params)

    # Evaluate the model on validation set
    y_val_pred = best_model.predict(X_val)
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    val_mae = mean_absolute_error(y_val, y_val_pred)

    print(f"Split {i + 1} Results:")
    print(f"  Best parameters: {best_params}")
    print(f"  Validation RMSE: {val_rmse}")
    print(f"  Validation MAE: {val_mae}")

    # Save the best model for the last split only
    if i == len(splits) - 1:
        best_model.save(os.path.join(exp_folder, 'best_lstm_model.keras'))
        print("Saved the best model from the last split.")

print()
print("All best hyperparameters across splits:", all_best_params)

Trial 28 Complete [00h 13m 09s]
val_loss: 0.7287741899490356

Best val_loss So Far: 0.5271056890487671
Total elapsed time: 05h 54m 16s

Search: Running Trial #29

Value             |Best Value So Far |Hyperparameter
3                 |2                 |num_lstm_layers
192               |32                |units
tanh              |relu              |activation
0.0004747         |0.0001045         |l2_regularizer
0.4               |0.2               |recurrent_dropout_0
1                 |3                 |num_dense_layers
128               |256               |dense_units_0
True              |False             |use_dense_dropout_0
rmsprop           |adam              |optimizer
mae               |mae               |loss
0.4               |0.4               |recurrent_dropout_1
0.2               |0.4               |recurrent_dropout_2
0.2               |0.3               |recurrent_dropout_3
128               |32                |dense_units_1
True              |True              |use_de